<a href="https://colab.research.google.com/github/JacobCHoward/comp215/blob/main/labs/lab02-review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2
----------------
#### Nam(s):
#### Date:

By the end of this lab you should be able to:
  * use strings, tuples, lists, and dictionaries (review)
  * use *f-string* to simplify string formatting operations
  * write functions and simple unit tests (review)
  * use *list comprehension* to implement map and filter algorithms


During this lab, you will be introduced to the following:
  * API Keys
  * JSON data formats
  * the `datetime` module to represent dates and times
  * the `pprint` module to format data structures

## Near Earth Objects

In this lab, we'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/) using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS).  But, as usual, let's do the imports first.


In [21]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

### dates & f-strings

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [22]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

2025-01-14 <class 'datetime.date'>
Today is: 2025-01-14


### Make an API Query

Let's get some data from the NEO database; here's a query that gets the observation "feed" for today.  Notice the first line of the block is an 'API key'.  An API key is a unique identifier that authenticates a user when making a request to an API.  Run the block below using the API key 'DEMO_KEY' (just to be sure it works), then go to https://api.nasa.gov and get your own API key.  Use your own API key for the remainder of the lab.

In [23]:
API_KEY = 'nCQMB1Od5gjMJFDMY850A63ZFghcrx4SQ4Tdqr8S'  # substitute your API key here

# get today's date (as a string)
today = str(datetime.date.today())

# use an f-string to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

# make the API request (recall from lab 1)
response = requests.request("GET", url, headers={}, data={})

# convert the response to json format (recall from lab 1)
data = json.loads(response.text)

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:1])

15 Near Earth Objects found for 2025-01-14
[{'absolute_magnitude_h': 20.74,
  'close_approach_data': [{'close_approach_date': '2025-01-14',
                           'close_approach_date_full': '2025-Jan-14 22:33',
                           'epoch_date_close_approach': 1736893980000,
                           'miss_distance': {'astronomical': '0.1161214726',
                                             'kilometers': '17371524.962223362',
                                             'lunar': '45.1712528414',
                                             'miles': '10794165.0794399156'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '96974.4703454129',
                                                 'kilometers_per_second': '26.9373528737',
                                                 'miles_per_hour': '60256.1646432372'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 1386.8357550659,
   

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [24]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


### Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.  We can use this `id` to fetch complete orbital and close approach data for the NEO.  For example, this query fetches the complete data set for the first hazardous asteroid...  Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".


In [25]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

{'absolute_magnitude_h': 20.74,
 'close_approach_data': [{'close_approach_date': '1901-04-18',
                          'close_approach_date_full': '1901-Apr-18 15:45',
                          'epoch_date_close_approach': -2168151300000,
                          'miss_distance': {'astronomical': '0.0421992908',
                                            'kilometers': '6312924.019190596',
                                            'lunar': '16.4155241212',
                                            'miles': '3922669.0889423848'},
                          'orbiting_body': 'Venus',
                          'relative_velocity': {'kilometers_per_hour': '130790.3819011481',
                                                'kilometers_per_second': '36.3306616392',
                                                'miles_per_hour': '81268.0570207439'}},
                         {'close_approach_date': '1912-07-16',
                          'close_approach_date_full': '1912-Jul-16 08:40'

## Exercise 1

In the code cell below, **re-write the accumulator loop** that creates the list of hazards as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [26]:
# Ex. 1 your code here
hazards =  [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.




## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [27]:
# Ex. 2 your code here
def closest_approach(approaches):
  '''Returns (date, miss km) of the closest approach to Earth within a dict of close approaches'''
  #Get the relevant part of the dict
  close_approaches = approaches['close_approach_data']
  #Arbitrary large number to serve as a starting point
  closest = 9999999999999999999999999999999

  for approach in close_approaches:
    if float(approach['miss_distance']['kilometers']) < closest and approach['orbiting_body']=='Earth':
      closest = float(approach['miss_distance']['kilometers'])
      date = approach['close_approach_date']
  return (date, closest)


#It wasn't until after I did this that I reread the question and saw it wanted the input to be a list. Oops.
#Thankfully after the first step closest_approach() is already built to handle a list.

def closest_approach_list(approaches):
  '''Returns (date, miss km) of the closest approach to Earth within a LIST of close approaches'''
   #Arbitrary large number to serve as a starting point
  closest = 9999999999999999999999999999999

  for approach in approaches:
    if float(approach['miss_distance']['kilometers']) < closest and approach['orbiting_body']=='Earth':
      closest = float(approach['miss_distance']['kilometers'])
      date = approach['close_approach_date']
  return (date, closest)

print(type(data['close_approach_data']))
print(closest_approach_list(data['close_approach_data']))


<class 'list'>
('1941-10-24', 5428129.197290249)


## Challenge Exercise

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [36]:
# Ex. 3 (challenge) your code here

#Fetch the week's list of NEO's.
API_KEY = 'nCQMB1Od5gjMJFDMY850A63ZFghcrx4SQ4Tdqr8S'
today = str(datetime.date.today())
yesterweek=str(datetime.date.today()-datetime.timedelta(days=7))
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={yesterweek}&end_date={today}&api_key={API_KEY}'

# Make the API request (recall from lab 1).
response = requests.request("GET", url, headers={}, data={})

# Convert the response to json format (recall from lab 1).
data = json.loads(response.text)

n_results = data['element_count']
neos = data['near_earth_objects']
print(f'{n_results} Near Earth Objects found from {yesterweek} through {today}')
pprint(neos)
# I pprinted it to check that it contains NEO's from the full week, but am
# omitting this from the final program for brevity's sake.

118 Near Earth Objects found from 2025-01-07 through 2025-01-14
{'2025-01-07': [{'absolute_magnitude_h': 20.2,
                 'close_approach_data': [{'close_approach_date': '2025-01-07',
                                          'close_approach_date_full': '2025-Jan-07 '
                                                                      '00:08',
                                          'epoch_date_close_approach': 1736208480000,
                                          'miss_distance': {'astronomical': '0.1236081659',
                                                            'kilometers': '18491518.333246633',
                                                            'lunar': '48.0835765351',
                                                            'miles': '11490096.6894162154'},
                                          'orbiting_body': 'Earth',
                                          'relative_velocity': {'kilometers_per_hour': '59576.37781234',
                    